In [2]:
!touch req.txt

In [3]:
!pip install easydict
!pip install addict
!pip install einops
!pip install tokenizers==0.20.3

!pip install transformers==4.46.3

# !pip install
# !pip install

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 34.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 34.8 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 1.0.0rc2
    Uninstalling huggingface-hub-1.0.0rc2:
      Successfully uninstalled huggingface-hub-1.0.0rc2
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.2
    Uninstalling tokenizers-0.21.2:
      Successfully uninstalled tokenizers-0.21.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 4.1.1 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
gradio 5.38.1 requires pydantic<2.12,>=2.0, but you have pydantic 2.12.0a1 which is incompatible.
transformers 4.53.3 requires tokenizers<0.22,>=0.21, but you have tokenizers 0.20.3 which is incomp

In [4]:
from transformers import AutoModel, AutoTokenizer
import torch
import os
model_name = 'deepseek-ai/DeepSeek-OCR'

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True, use_safetensors=True)
model = model.eval().cuda().to(torch.bfloat16)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/801 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

modeling_deepseekocr.py: 0.00B [00:00, ?B/s]

...

modeling_deepseekv2.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/deepseek-ai/DeepSeek-OCR:
- modeling_deepseekv2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/deepseek-ai/DeepSeek-OCR:
- modeling_deepseekocr.py
- deepencoder.py
- configuration_deepseek_v2.py
- conversation.py
- modeling_deepseekv2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
2025-11-03 14:27:38.905827: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762180059.135421      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDN

model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-000001.safetensors:   0%|          | 0.00/6.67G [00:00<?, ?B/s]

Some weights of DeepseekOCRForCausalLM were not initialized from the model checkpoint at deepseek-ai/DeepSeek-OCR and are newly initialized: ['model.vision_model.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
from transformers import TextStreamer

class NoEOSTextStreamer(TextStreamer):
    def on_finalized_text(self, text: str, stream_end: bool = False):

        eos_text = self.tokenizer.decode([self.tokenizer.eos_token_id], skip_special_tokens=False)
        text = text.replace(eos_text, "\n")
        print(text, flush=True, end="")

In [9]:
"""
From https://github.com/lm-sys/FastChat/blob/main/fastchat/conversation.py
"""

import dataclasses
from enum import IntEnum, auto
from typing import Any, Dict, List


class SeparatorStyle(IntEnum):
    """Separator styles."""

    DeepSeek = auto()
    DeepSeekV2 = auto()
    PLAIN = auto()
    ALIGNMENT = auto()


@dataclasses.dataclass
class Conversation:
    """A class that manages prompt templates and keeps all conversation history."""

    # The name of this template
    name: str
    # The template of the system prompt
    system_template: str = "{system_message}"
    # The system message
    system_message: str = ""
    # The names of two roles
    roles: List[str] = (("USER", "ASSISTANT"),)
    # All messages. Each item is (role, message).
    messages: List[List[str]] = ()
    # The number of few shot examples
    offset: int = 0
    # The separator style and configurations
    sep_style: SeparatorStyle = SeparatorStyle.DeepSeek
    sep: str = "\n"
    sep2: str = None
    # Stop criteria (the default one is EOS token)
    stop_str: str = None
    # Stops generation if meeting any token in this list
    stop_token_ids: List[int] = None

    def get_prompt(self) -> str:
        """Get the prompt for generation."""
        system_prompt = self.system_template.format(system_message=self.system_message)
        if self.sep_style == SeparatorStyle.DeepSeek:
            seps = [self.sep, self.sep2]
            if system_prompt == "" or system_prompt is None:
                ret = ""
            else:
                ret = system_prompt + seps[0]
            for i, (role, message) in enumerate(self.messages):
                if message:
                    ret += role + ": " + message + seps[i % 2]
                else:
                    ret += role + ":"
            return ret
        elif self.sep_style == SeparatorStyle.DeepSeekV2:
            seps = [self.sep, self.sep2]
            if system_prompt == "" or system_prompt is None:
                ret = ""
            else:
                ret = system_prompt + seps[0]
            for i, (role, message) in enumerate(self.messages):
                if message:
                    if role == "User":
                        ret += "<｜sft▁begin｜>\n" + message + self.sep #<｜sft▁begin｜>User Input<｜sft▁end｜>\nResponse<｜end▁of▁sentence｜>
                    else:
                        ret += message + self.sep2
                else:
                    ret = ret
            return ret

        elif self.sep_style == SeparatorStyle.PLAIN:
            seps = [self.sep, self.sep2]
            ret = ""
            for i, (role, message) in enumerate(self.messages):
                if message:
                    if type(message) is tuple:
                        message, _, _ = message
                    if i % 2 == 0:
                        ret += message + seps[i % 2]
                    else:
                        ret += message + seps[i % 2]
                else:
                    ret += ""
            return ret
        elif self.sep_style == SeparatorStyle.ALIGNMENT:
            seps = [self.sep, self.sep2]
            ret = ""
            for i, (role, message) in enumerate(self.messages):
                if message:
                    if type(message) is tuple:
                        message, _, _ = message
                    if i % 2 == 0:
                        ret += '<image>\n' + seps[i % 2]
                    else:
                        ret += message + seps[i % 2]
                else:
                    ret += ""
            return ret
        else:
            raise ValueError(f"Invalid style: {self.sep_style}")

    def set_system_message(self, system_message: str):
        """Set the system message."""
        self.system_message = system_message

    def append_message(self, role: str, message: str):
        """Append a new message."""
        self.messages.append([role, message])

    def update_last_message(self, message: str):
        """Update the last output.

        The last message is typically set to be None when constructing the prompt,
        so we need to update it in-place after getting the response from a model.
        """
        self.messages[-1][1] = message

    def reset_message(self):
        """Reset a new message."""
        self.messages = []

    def to_gradio_chatbot(self):
        """Convert the conversation to gradio chatbot format."""
        ret = []
        for i, (role, msg) in enumerate(self.messages[self.offset :]):
            if i % 2 == 0:
                ret.append([msg, None])
            else:
                ret[-1][-1] = msg
        return ret

    def to_openai_api_messages(self):
        """Convert the conversation to OpenAI chat completion format."""
        system_prompt = self.system_template.format(system_message=self.system_message)
        ret = [{"role": "system", "content": system_prompt}]

        for i, (_, msg) in enumerate(self.messages[self.offset :]):
            if i % 2 == 0:
                ret.append({"role": "user", "content": msg})
            else:
                if msg is not None:
                    ret.append({"role": "assistant", "content": msg})
        return ret

    def copy(self):
        return Conversation(
            name=self.name,
            system_template=self.system_template,
            system_message=self.system_message,
            roles=self.roles,
            messages=[[x, y] for x, y in self.messages],
            offset=self.offset,
            sep_style=self.sep_style,
            sep=self.sep,
            sep2=self.sep2,
            stop_str=self.stop_str,
            stop_token_ids=self.stop_token_ids,
        )

    def dict(self):
        return {
            "template_name": self.name,
            "system_message": self.system_message,
            "roles": self.roles,
            "messages": self.messages,
            "offset": self.offset,
        }


# A global registry for all conversation templates
conv_templates: Dict[str, Conversation] = {}


def register_conv_template(template: Conversation, override: bool = False):
    """Register a new conversation template."""
    if not override:
        assert template.name not in conv_templates, f"{template.name} has been registered."

    conv_templates[template.name] = template


def get_conv_template(name: str) -> Conversation:
    """Get a conversation template."""
    return conv_templates[name].copy()


register_conv_template(
    Conversation(
        name="deepseek",
        system_template="{system_message}",
        # system_message="You are a helpful assistant. Please answer truthfully and write out your "
        # "thinking step by step to be sure you get the right answer.",
        system_message="",
        roles=("<|User|>", "<|Assistant|>"),
        messages=(),
        offset=0,
        sep_style=SeparatorStyle.DeepSeek,
        sep="\n\n",
        sep2="<｜end▁of▁sentence｜>",
        stop_token_ids=[100001],
        stop_str=["User:", "<｜end▁of▁sentence｜>"]
    )
)
register_conv_template(
    Conversation(
        name="deepseekv2",
        system_template="{system_message}",
        # system_message="You are a helpful assistant. Please answer truthfully and write out your "
        # "thinking step by step to be sure you get the right answer.",
        system_message="",
        roles=("<｜User｜>", "<｜Assistant｜>"),
        messages=(),
        offset=0,
        sep_style=SeparatorStyle.DeepSeek,
        sep="",
        sep2="<｜end▁of▁sentence｜>",
        stop_token_ids=[100001],
        stop_str=["User:", "<｜end▁of▁sentence｜>"]
    )
)


register_conv_template(
    Conversation(
        name="plain",
        system_template="",
        system_message="",
        roles=("", ""),
        messages=(),
        offset=0,
        sep_style=SeparatorStyle.PLAIN,
        sep="",
        sep2="",
        stop_token_ids=[100001],
        stop_str=['</s>'],
    )
)


register_conv_template(
    Conversation(
        name="alignment",
        system_template="",
        system_message="",
        roles=("", ""),
        messages=(),
        offset=0,
        sep_style=SeparatorStyle.ALIGNMENT,
        sep="",
        sep2="",
        stop_token_ids=[100001],
        stop_str=['</s>'],
    )
)

In [29]:
def normalize_transform(mean, std):
    if mean is None and std is None:
        transform = None
    elif mean is None and std is not None:
        mean = [0.] * len(std)
        transform = transforms.Normalize(mean=mean, std=std)
    elif mean is not None and std is None:
        std = [1.] * len(mean)
        transform = transforms.Normalize(mean=mean, std=std)
    else:
        transform = transforms.Normalize(mean=mean, std=std)

    return transform

In [13]:
from typing import List, Optional, Tuple, Union
class BasicImageTransform:
    def __init__(
        self, 
        mean: Optional[Tuple[float, float, float]] = (0.5, 0.5, 0.5),
        std: Optional[Tuple[float, float, float]] = (0.5, 0.5, 0.5),
        normalize: bool = True
    ):
        self.mean = mean
        self.std = std
    
        transform_pipelines = [
            transforms.ToTensor()
        ]

        normalize = normalize_transform(mean, std) if normalize else nn.Identity()
        if normalize is not None:
            transform_pipelines.append(normalize)

        self.transform = transforms.Compose(transform_pipelines)
    
    def __call__(self, x):
        x = self.transform(x)
        return x

In [32]:
import os
import math
import torch
from PIL import Image, ImageOps
from typing import List, Dict
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from tqdm import tqdm

def draw_bounding_boxes(image, refs, ouput_path):

    image_width, image_height = image.size
    
    img_draw = image.copy()
    draw = ImageDraw.Draw(img_draw)

    overlay = Image.new('RGBA', img_draw.size, (0, 0, 0, 0))
    draw2 = ImageDraw.Draw(overlay)
    
    # try:
    # except IOError:
    #     try:
    #         font = ImageFont.truetype("DejaVuSans.ttf", 20) 
    #     except IOError:
    font = ImageFont.load_default()

    img_idx = 0
    
    for i, ref in enumerate(refs):
        try:
            result = extract_coordinates_and_label(ref, image_width, image_height)
            if result:
                label_type, points_list = result
                
                color = (np.random.randint(0, 200), np.random.randint(0, 200), np.random.randint(0, 255))

                color_a = color + (20, )
                for points in points_list:
                    x1, y1, x2, y2 = points

                    x1 = int(x1 / 999 * image_width)
                    y1 = int(y1 / 999 * image_height)

                    x2 = int(x2 / 999 * image_width)
                    y2 = int(y2 / 999 * image_height)

                    if label_type == 'image':
                        try:
                            cropped = image.crop((x1, y1, x2, y2))
                            cropped.save(f"{ouput_path}/images/{img_idx}.jpg")
                        except Exception as e:
                            print(e)
                            pass
                        img_idx += 1
                        
                    try:
                        if label_type == 'title':
                            draw.rectangle([x1, y1, x2, y2], outline=color, width=4)
                            draw2.rectangle([x1, y1, x2, y2], fill=color_a, outline=(0, 0, 0, 0), width=1)
                        else:
                            draw.rectangle([x1, y1, x2, y2], outline=color, width=2)
                            draw2.rectangle([x1, y1, x2, y2], fill=color_a, outline=(0, 0, 0, 0), width=1)
                        text_x = x1
                        text_y = max(0, y1 - 15)
                            
                        
                        text_bbox = draw.textbbox((0, 0), label_type, font=font)
                        text_width = text_bbox[2] - text_bbox[0]
                        text_height = text_bbox[3] - text_bbox[1]
                        draw.rectangle([text_x, text_y, text_x + text_width, text_y + text_height], 
                                    fill=(255, 255, 255, 30))
                        
                        draw.text((text_x, text_y), label_type, font=font, fill=color)
                    except:
                        pass
        except:
            continue
    img_draw.paste(overlay, (0, 0), overlay)
    return img_draw

def format_messages(
        conversations: List[Dict[str, str]],
        sft_format: str = "deepseek",
        system_prompt: str = "",
):
    """
    Applies the SFT template to conversation.
    Args:
        conversations (List[Dict]): A List of messages.
        sft_format (str, optional): The format of the SFT template to use. Defaults to "deepseek".
        system_prompt (str, optional): The system prompt to use in the SFT template. Defaults to "".
    Returns:
        sft_prompt (str): The formatted text.
    """

    conv = get_conv_template(sft_format)
    conv.set_system_message(system_prompt)
    for message in conversations:
        conv.append_message(message["role"], message["content"].strip())
    sft_prompt = conv.get_prompt().strip()

    return sft_prompt

def find_closest_aspect_ratio(aspect_ratio, target_ratios, width, height, image_size):
    best_ratio_diff = float('inf')
    best_ratio = (1, 1)
    area = width * height
    for ratio in target_ratios:
        target_aspect_ratio = ratio[0] / ratio[1]
        ratio_diff = abs(aspect_ratio - target_aspect_ratio)
        if ratio_diff < best_ratio_diff:
            best_ratio_diff = ratio_diff
            best_ratio = ratio
        elif ratio_diff == best_ratio_diff:
            if area > 0.5 * image_size * image_size * ratio[0] * ratio[1]:
                best_ratio = ratio
    # print(f'width: {width}, height: {height}, best_ratio: {best_ratio}')
    return best_ratio

def dynamic_preprocess(image, min_num=2, max_num=9, image_size=640, use_thumbnail=False):
    orig_width, orig_height = image.size
    aspect_ratio = orig_width / orig_height

    # calculate the existing image aspect ratio
    target_ratios = set(
        (i, j) for n in range(min_num, max_num + 1) for i in range(1, n + 1) for j in range(1, n + 1) if
        i * j <= max_num and i * j >= min_num)
    # print(target_ratios)
    target_ratios = sorted(target_ratios, key=lambda x: x[0] * x[1])

    # find the closest aspect ratio to the target
    target_aspect_ratio = find_closest_aspect_ratio(
        aspect_ratio, target_ratios, orig_width, orig_height, image_size)

    # print(target_aspect_ratio)
    # calculate the target width and height
    target_width = image_size * target_aspect_ratio[0]
    target_height = image_size * target_aspect_ratio[1]
    blocks = target_aspect_ratio[0] * target_aspect_ratio[1]

    # resize the image
    resized_img = image.resize((target_width, target_height))
    processed_images = []
    for i in range(blocks):
        box = (
            (i % (target_width // image_size)) * image_size,
            (i // (target_width // image_size)) * image_size,
            ((i % (target_width // image_size)) + 1) * image_size,
            ((i // (target_width // image_size)) + 1) * image_size
        )
        # split the image
        split_img = resized_img.crop(box)
        processed_images.append(split_img)
    assert len(processed_images) == blocks
    if use_thumbnail and len(processed_images) != 1:
        thumbnail_img = image.resize((image_size, image_size))
        processed_images.append(thumbnail_img)
    return processed_images, target_aspect_ratio
    
def text_encode(tokenizer, text: str, bos: bool = True, eos: bool = False):
    t = tokenizer.encode(text, add_special_tokens=False)
    bos_id = 0
    eos_id = 1
    if bos:
        t = [bos_id] + t
    if eos:
        t = t + [eos_id]

    return t
    
def infer_with_image_object(
    model,  # Pass the instantiated model object here
    tokenizer,
    image_object: Image.Image,
    prompt: str = '<image>\n<|grounding|>Convert the document to markdown. ',
    output_path: str = '',
    base_size: int = 1024,
    image_size: int = 640,
    crop_mode: bool = True,
    test_compress: bool = False,
    save_results: bool = False,
    eval_mode: bool = True
):
    """
    Mimics the behavior of the DeepseekOCRForCausalLM.infer method but
    operates as an external function and accepts a PIL Image object.
    """
    
    # 1. Disable torch init (was self.disable_torch_init())
    model.disable_torch_init()

    # 2. Setup directories
    os.makedirs(output_path, exist_ok=True)
    os.makedirs(f'{output_path}/images', exist_ok=True)

    # 3. Setup conversation and images list
    if not prompt:
        assert False, f'prompt is none!'
    
    images = []
    image_draw = None
    conversation = [
        {
            "role": "<|User|>",
            # "content": "<image>\n<|grounding|>Given the layout of the image. ",
            "content": f'{prompt}',
            # "content": "君不见黄河之水天上来的下一句是什么？",
            # "content": "<image>\nFree OCR. ",
            # "content": "<image>\nParse the figure. ",
            # "content": "<image>\nExtract the text in the image. ",
            # "images": [f'{image_file}'],
        },
        {"role": "<|Assistant|>", "content": ""},
    ]


    # 4. Format prompt
    prompt = format_messages(conversations=conversation, sft_format='plain', system_prompt='')
    patch_size = 16
    downsample_ratio = 4
    images = [image_object]

    # 5. Get image properties (if image exists)
    valid_img_tokens = 0
    ratio = 1
    if image_draw:
        w, h = image_draw.size
        ratio = 1 - ((max(w, h) - min(w, h)) / (max(w, h)))
    
    # 6. Initialize transforms and tokens
    image_transform = BasicImageTransform(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5), normalize=True)
    images_seq_mask = []

    image_token = '<image>'
    image_token_id = 128815
    text_splits = prompt.split(image_token)

    images_list, images_crop_list, images_seq_mask = [], [], []
    tokenized_str = []
    images_spatial_crop = []
    for text_sep, image in zip(text_splits, images):
            tokenized_sep = text_encode(tokenizer, text_sep, bos=False, eos=False)

            tokenized_str += tokenized_sep
            images_seq_mask += [False] * len(tokenized_sep)

            if crop_mode:

                if image.size[0] <= 640 and image.size[1] <= 640:
                    crop_ratio = [1, 1]

                else:
                    if crop_mode:
                        # best_width, best_height = select_best_resolution(image.size, self.candidate_resolutions)
                        images_crop_raw, crop_ratio = dynamic_preprocess(image)
                    else:
                        # best_width, best_height = self.image_size, self.image_size
                        crop_ratio = [1, 1]
                
                """process the global view"""
                # image = image.resize((base_size, base_size))
                global_view = ImageOps.pad(image, (base_size, base_size),
                                        color=tuple(int(x * 255) for x in image_transform.mean))
                
                if base_size == 1024:
                    valid_img_tokens += int(256 * ratio)
                elif base_size == 1280:
                    valid_img_tokens += int(400 * ratio)
                
                images_list.append(image_transform(global_view).to(torch.bfloat16))

                # global_view_tensor = image_transform(global_view).to(torch.bfloat16)

                width_crop_num, height_crop_num = crop_ratio

                images_spatial_crop.append([width_crop_num, height_crop_num])
                
                
                if width_crop_num > 1 or height_crop_num > 1:
                    """process the local views"""
                    
                    for i in range(len(images_crop_raw)):
                        images_crop_list.append(image_transform(images_crop_raw[i]).to(torch.bfloat16))
                
                if image_size == 640:
                    valid_img_tokens += len(images_crop_list) * 100

                num_queries = math.ceil((image_size // patch_size) / downsample_ratio)
                num_queries_base = math.ceil((base_size // patch_size) / downsample_ratio)
                tokenized_image = ([image_token_id] * num_queries_base + [image_token_id]) * num_queries_base
                tokenized_image += [image_token_id]
                if width_crop_num > 1 or height_crop_num > 1:
                    tokenized_image += ([image_token_id] * (num_queries * width_crop_num) + [image_token_id]) * (
                                num_queries * height_crop_num)
                tokenized_str += tokenized_image
                images_seq_mask += [True] * len(tokenized_image)
                # num_image_tokens.append(len(tokenized_image))

            else:
                # best_width, best_height = self.image_size, self.image_size
                # print(image.size, (best_width, best_height)) # check the select_best_resolutions func

                """process the global view"""
                if image_size <= 640:
                    print('directly resize')
                    image = image.resize((image_size, image_size))
                # else:
                global_view = ImageOps.pad(image, (image_size, image_size),
                                        color=tuple(int(x * 255) for x in image_transform.mean))
                images_list.append(image_transform(global_view).to(torch.bfloat16))

                if base_size == 1024:
                    valid_img_tokens += int(256 * ratio)
                elif base_size == 1280:
                    valid_img_tokens += int(400 * ratio)
                elif base_size == 640:
                    valid_img_tokens += int(100 * 1)
                elif base_size == 512:
                    valid_img_tokens += int(64 * 1)

                width_crop_num, height_crop_num = 1, 1

                images_spatial_crop.append([width_crop_num, height_crop_num])


                """add image tokens"""
                num_queries = math.ceil((image_size // patch_size) / downsample_ratio)

                tokenized_image = ([image_token_id] * num_queries + [image_token_id]) * num_queries
                tokenized_image += [image_token_id]
                # tokenized_image += ([self.image_token_id] * (num_queries * width_crop_num) + [self.image_token_id]) * (
                #             num_queries * height_crop_num)
                tokenized_str += tokenized_image
                images_seq_mask += [True] * len(tokenized_image)
                # num_image_tokens.append(len(tokenized_image))
        

            tokenized_sep = text_encode(tokenizer, text_splits[-1], bos=False, eos=False)
            tokenized_str += tokenized_sep
            images_seq_mask += [False] * len(tokenized_sep)

            """add the bos tokens"""
            bos_id = 0
            tokenized_str = [bos_id] + tokenized_str 
            images_seq_mask = [False] + images_seq_mask
            input_ids = torch.LongTensor(tokenized_str)
            images_seq_mask = torch.tensor(images_seq_mask, dtype=torch.bool)

            if len(images_list) == 0:
                images_ori = torch.zeros((1, 3, image_size, image_size))
                images_spatial_crop = torch.zeros((1, 2), dtype=torch.long)
                images_crop = torch.zeros((1, 3, base_size, base_size))

            else:
                images_ori = torch.stack(images_list, dim=0)
                images_spatial_crop = torch.tensor(images_spatial_crop, dtype=torch.long)
                if images_crop_list:
                    images_crop = torch.stack(images_crop_list, dim=0)
                else:
                    images_crop = torch.zeros((1, 3, base_size, base_size))



            if not eval_mode:
                streamer = NoEOSTextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=False)
                with torch.autocast("cuda", dtype=torch.bfloat16):
                    with torch.no_grad():
                        output_ids = model.generate(
                            input_ids.unsqueeze(0).cuda(),
                            images=[(images_crop.cuda(), images_ori.cuda())],
                            images_seq_mask = images_seq_mask.unsqueeze(0).cuda(),
                            images_spatial_crop = images_spatial_crop,
                            temperature=0.0,
                            eos_token_id=tokenizer.eos_token_id,
                            streamer=streamer,
                            max_new_tokens=8192,
                            no_repeat_ngram_size = 20,
                            use_cache = True
                            )

            else:
                with torch.autocast("cuda", dtype=torch.bfloat16):
                    with torch.no_grad():
                        output_ids = model.generate(
                            input_ids.unsqueeze(0).cuda(),
                            images=[(images_crop.cuda(), images_ori.cuda())],
                            images_seq_mask = images_seq_mask.unsqueeze(0).cuda(),
                            images_spatial_crop = images_spatial_crop,
                            temperature=0.0,
                            eos_token_id=tokenizer.eos_token_id,
                            max_new_tokens=8192,
                            no_repeat_ngram_size = 35,
                            use_cache = True
                            )
                    

            if '<image>' in conversation[0]['content'] and eval_mode:
                    outputs = tokenizer.decode(output_ids[0, input_ids.unsqueeze(0).cuda().shape[1]:])
                    stop_str = '<｜end▁of▁sentence｜>'
                    if outputs.endswith(stop_str):
                        outputs = outputs[:-len(stop_str)]
                    # re_match
                    outputs = outputs.strip()

                    return outputs
            
            if '<image>' in conversation[0]['content'] and test_compress:
                outputs = tokenizer.decode(output_ids[0, input_ids.unsqueeze(0).cuda().shape[1]:])
                pure_texts_outputs_token_length = len(text_encode(tokenizer, outputs, bos=False, eos=False))
                print('='*50)
                print('image size: ', (w, h))
                print('valid image tokens: ', int(valid_img_tokens))
                print('output texts tokens (valid): ', pure_texts_outputs_token_length)
                print('compression ratio: ', round(pure_texts_outputs_token_length/valid_img_tokens, 2))
                print('='*50)


            if '<image>' in conversation[0]['content'] and save_results:
                outputs = tokenizer.decode(output_ids[0, input_ids.unsqueeze(0).cuda().shape[1]:])
                stop_str = '<｜end▁of▁sentence｜>'

                print('='*15 + 'save results:' + '='*15)
                
                # # # # conv.messages[-1][-1] = outputs
                if outputs.endswith(stop_str):
                    outputs = outputs[:-len(stop_str)]
                outputs = outputs.strip()

                matches_ref, matches_images, mathes_other = re_match(outputs)
                # print(matches_ref)
                result = draw_bounding_boxes(image_draw, matches_ref, output_path)



                for idx, a_match_image in enumerate(tqdm(matches_images, desc="image")):
                    outputs = outputs.replace(a_match_image, '![](images/' + str(idx) + '.jpg)\n')
                
                for idx, a_match_other in enumerate(tqdm(mathes_other, desc="other")):
                    outputs = outputs.replace(a_match_other, '').replace('\\coloneqq', ':=').replace('\\eqqcolon', '=:')


                # if 'structural formula' in conversation[0]['content']:
                #     outputs = '<smiles>' + outputs + '</smiles>'
                with open(f'{output_path}/result.mmd', 'w', encoding = 'utf-8') as afile:
                    afile.write(outputs)

                if 'line_type' in outputs:
                    import matplotlib.pyplot as plt
                    lines = eval(outputs)['Line']['line']

                    line_type = eval(outputs)['Line']['line_type']
                    # print(lines)

                    endpoints = eval(outputs)['Line']['line_endpoint']

                    fig, ax = plt.subplots(figsize=(3,3), dpi=200)
                    ax.set_xlim(-15, 15)
                    ax.set_ylim(-15, 15)

                    for idx, line in enumerate(lines):
                        try:
                            p0 = eval(line.split(' -- ')[0])
                            p1 = eval(line.split(' -- ')[-1])

                            if line_type[idx] == '--':
                                ax.plot([p0[0], p1[0]], [p0[1], p1[1]], linewidth=0.8, color='k')
                            else:
                                ax.plot([p0[0], p1[0]], [p0[1], p1[1]], linewidth = 0.8, color = 'k')

                            ax.scatter(p0[0], p0[1], s=5, color = 'k')
                            ax.scatter(p1[0], p1[1], s=5, color = 'k')
                        except:
                            pass

                    for endpoint in endpoints:

                        label = endpoint.split(': ')[0]
                        (x, y) = eval(endpoint.split(': ')[1])
                        ax.annotate(label, (x, y), xytext=(1, 1), textcoords='offset points', 
                                    fontsize=5, fontweight='light')
                    

                    plt.savefig(f'{output_path}/geo.jpg')
                    plt.close()

                result.save(f"{output_path}/result_with_boxes.jpg")

In [16]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("NAMAA-Space/QariOCR-v0.3-markdown-mixed-dataset", split="test")

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/25.2M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/3.08M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/3.18M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/29563 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3696 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3695 [00:00<?, ? examples/s]

In [17]:
image_file = ds['image'][0]

In [27]:
from torchvision import transforms
from torchvision.transforms.functional import InterpolationMode

In [21]:
import requests
from io import BytesIO

response = requests.get(image_file)
pil_image = Image.open(BytesIO(response.content)).convert("RGB")

In [33]:
a = infer_with_image_object(model, tokenizer, pil_image, output_path="deepseek_eval")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([6, 100, 1280])


In [34]:
a

'<|ref|>text<|/ref|><|det|>[[160, 106, 875, 343]]<|/det|>\nتلقت المغنية فاطمة الزهراء العروسي، رسالة من الملك محمد السادس، بعد تقديمها لأغنية «على سلامة سيدنا» إهداء للملك، بمناسبة شفائه بعد إجرائه لعملية جراحية، والتي تكالت بالنجاح ونشرت العروسي نص الرسالة على صفحاتها الرسمية على مواقع التواصل، حيث وجهت الشكر للملك على الرسالة مؤكدة على أنها تركت لديها أثرا عميقا وبرهجة وسرور، وأن الإلتفافة ستكون حافزا لديها لتقديم المزيد من العطاء والإجتهاد \n\n<|ref|>text<|/ref|><|det|>[[469, 373, 864, 421]]<|/det|>\n© حقوق النشر: DR'

In [35]:
import re

def extract_clean_text(input_str):
    # Remove all <|...|> tags and whitespace-only lines
    # Keep only lines that contain actual text (non-tag content)
    lines = input_str.strip().split('\n')
    clean_lines = []
    for line in lines:
        # Strip leading/trailing whitespace
        if line.strip().startswith("<|"):
          continue
        stripped = line.strip()
        # Skip empty lines
        if not stripped:
            continue
        # Remove all <|...|> style tags
        cleaned = re.sub(r'<\|[^|]*\|>', '', stripped)
        # If after removing tags, there's still meaningful text, keep it
        if cleaned.strip():
            clean_lines.append(cleaned.strip())


    return re.sub('<｜end▁of▁sentence｜>', '', '\n'.join(clean_lines))

In [36]:
extract_clean_text(a)

'تلقت المغنية فاطمة الزهراء العروسي، رسالة من الملك محمد السادس، بعد تقديمها لأغنية «على سلامة سيدنا» إهداء للملك، بمناسبة شفائه بعد إجرائه لعملية جراحية، والتي تكالت بالنجاح ونشرت العروسي نص الرسالة على صفحاتها الرسمية على مواقع التواصل، حيث وجهت الشكر للملك على الرسالة مؤكدة على أنها تركت لديها أثرا عميقا وبرهجة وسرور، وأن الإلتفافة ستكون حافزا لديها لتقديم المزيد من العطاء والإجتهاد\n© حقوق النشر: DR'

In [42]:
re.sub(r'<[^>]+>','',ds['text'][0])

'تلقت المغنية فاطمة الزهراء العروسي، رسالة من الملك محمد السادس، بعد تقديمها لأغنية «على سلامة سيدنا » إهداء للملك، بمناسبة شفائه بعد إجرائه لعميلة جراحية، والتي تكللت بالنجاح\nونشرت العروسي نص الرسالة على صفحاتها الرسمية على مواقع التواصل، حيث وجهت الشكر للملك على الرسالة مؤكدة على أنها تركت لديها أثرا عميقا وبهجة وسرور، وأن الإلتفافة ستكون حافزا لديها لتقديم المزيد من العطاء والإجتهاد\n© حقوق النشر : DR'